#  Neural Networks: Regression on House Pricing Dataset
We consider a reduced version of a dataset containing house sale prices for King County, which includes Seattle. It includes homes sold between May 2014 and May 2015.

https://www.kaggle.com/harlfoxem/housesalesprediction

For each house we know 18 house features (e.g., number of bedrooms, number of bathrooms, etc.) plus its price, that is what we would like to predict.

In [1]:
#put here your ``numero di matricola''
numero_di_matricola = 2074282 # COMPLETE

## Insert your ID number ("numero di matricola") below

In [2]:
#put here your ``numero di matricola''
numero_di_matricola = 2074282 # COMPLETE

In [3]:
#import all packages needed
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

Load the data, remove data samples/points with missing values (NaN) and take a look at them.

In [4]:
#load the data
df = pd.read_csv('kc_house_data.csv', sep = ',')

#remove the data samples with missing values (NaN)
df = df.dropna() 

df.describe()

,id,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
count,3.164000e+03,3.164000e+03,3164.000000,3164.000000,3164.000000,3.164000e+03,3164.000000,3164.000000,3164.000000,3164.000000,3164.000000,3164.000000,3164.000000,3164.000000,3164.000000,3164.000000,3164.000000,3164.000000,3164.000000,3164.000000
mean,4.645240e+09,5.354358e+05,3.381163,2.071903,2070.027813,1.525054e+04,1.434893,0.009798,0.244311,3.459229,7.615676,1761.252212,308.775601,1967.489254,94.668774,98077.125158,47.557868,-122.212337,1982.544564,13176.302465
std,2.854203e+09,3.809004e+05,0.895472,0.768212,920.251879,4.254457e+04,0.507792,0.098513,0.776298,0.682592,1.166324,815.934864,458.977904,28.095275,424.439427,54.172937,0.140789,0.139577,686.256670,25413.180755
min,1.000102e+06,7.500000e+04,0.000000,0.000000,380.000000,6.490000e+02,1.000000,0.000000,0.000000,1.000000,3.000000,380.000000,0.000000,1900.000000,0.000000,98001.000000,47.177500,-122.514000,620.000000,660.000000
25%,2.199775e+09,3.150000e+05,3.000000,1.500000,1430.000000,5.453750e+03,1.000000,0.000000,0.000000,3.000000,7.000000,1190.000000,0.000000,1950.000000,0.000000,98032.000000,47.459575,-122.324250,1480.000000,5429.500000
50%,4.027701e+09,4.450000e+05,3.000000,2.000000,1910.000000,8.000000e+03,1.000000,0.000000,0.000000,3.000000,7.000000,1545.000000,0.000000,1969.000000,0.000000,98059.000000,47.572500,-122.226000,1830.000000,7873.000000
75%,7.358175e+09,6.402500e+05,4.000000,2.500000,2500.000000,1.122250e+04,2.000000,0.000000,0.000000,4.000000,8.000000,2150.000000,600.000000,1990.000000,0.000000,98117.000000,47.680250,-122.124000,2360.000000,10408.250000
max,9.839301e+09,5.350000e+06,8.000000,6.000000,8010.000000,1.651359e+06,3.500000,1.000000,4.000000,5.000000,12.000000,6720.000000,2620.000000,2015.000000,2015.000000,98199.000000,47.777600,-121.315000,5790.000000,425581.000000


Extract input and output data. We want to predict the price by using features other than id as input.

In [5]:
Data = df.values
# m = number of input samples
m = Data.shape[0]
print("Amount of data:",m)
Y = Data[:m,2]
X = Data[:m,3:]

Amount of data: 3164


## Data Pre-Processing

We split the data into 3 parts: one will be used for training and choosing the parameters, one for choosing among different models, and one for testing. The part for training and choosing the parameters will consist of $2/3$ of all samples, the one for choosing among different models will consist of $1/6$ of all samples, while the other part consists of the remaining $1/6$-th of all samples.

In [6]:
# Split data into train (2/3 of samples), validation (1/6 of samples), and test data (the rest)
m_train = int(2./3.*m)
m_val = int((m-m_train)/2.)
m_test = m - m_train - m_val
print("Amount of data for training and deciding parameters:",m_train)
print("Amount of data for validation (choosing among different models):",m_val)
print("Amount of data for test:",m_test)
from sklearn.model_selection import train_test_split

Xtrain_and_val, Xtest, Ytrain_and_val, Ytest = train_test_split(X, Y, test_size=m_test/m, random_state=numero_di_matricola)
Xtrain, Xval, Ytrain, Yval = train_test_split(Xtrain_and_val, Ytrain_and_val, test_size=m_val/(m_train+m_val), random_state=numero_di_matricola)

Amount of data for training and deciding parameters: 2109
Amount of data for validation (choosing among different models): 527
Amount of data for test: 528


Let's standardize the data.

In [7]:
# Data pre-processing
from sklearn import preprocessing
scaler = preprocessing.StandardScaler().fit(Xtrain)
Xtrain_scaled = scaler.transform(Xtrain)
Xtrain_and_val_scaled = scaler.transform(Xtrain_and_val)
Xval_scaled = scaler.transform(Xval)
Xtest_scaled = scaler.transform(Xtest)

## Neural Networks
Let's start by learning a simple neural network with 1 hidden node.
Note: we are going to use the input parameter solver='lbfgs' and random_state=numero_di_matricola to fix the random seed (so results are reproducible).

In [8]:
#let's load the MLPRegressor

from sklearn.neural_network import MLPRegressor

#let's define the model
mlp = MLPRegressor(hidden_layer_sizes=(1, ), solver='lbfgs', random_state=numero_di_matricola )

#let's learn the model on training data
mlp.fit(Xtrain_scaled, Ytrain)

#let's print the error (1 - R^2) on training data
print("Training error: ", 1. - mlp.score(Xtrain_scaled, Ytrain))

#let's print the error (1 - R^2) on validation data
print("Validation error: ", 1. - mlp.score(Xval_scaled, Yval))

#let's print the coefficients of the model for the input nodes (but not the bias)
print(mlp.coefs_)

#let's print the coefficient for the bias (i.e., the bias)
print(mlp.intercepts_)

Training error:  0.29042834117041194
Validation error:  0.25859709671417375
[array([[ -65.1561539 ],
       [  91.88705386],
       [ 134.18897051],
       [  30.74112316],
       [ -24.21412318],
       [ 123.86260491],
       [  98.16222198],
       [  26.27383072],
       [ 194.11940951],
       [ 137.28989476],
       [  24.87813579],
       [-156.2503569 ],
       [  -1.72604897],
       [ -46.69720186],
       [ 154.2014421 ],
       [ -42.89073063],
       [  57.38216095],
       [ -40.796412  ]]), array([[554.30331639]])]
[array([961.29608923]), array([537.39806167])]


## Neural Networks vs Linear Models

Let's learn a linear model on the other same data and compare the results with the simple NN above.

In [9]:
from sklearn import linear_model

LR = linear_model.LinearRegression()

LR.fit(Xtrain_scaled, Ytrain)

print("Training error: ", 1. - LR.score(Xtrain_scaled, Ytrain))
print("Validation error: ", 1. - LR.score(Xval_scaled, Yval))

print(LR.coef_)
print(LR.intercept_)

Training error:  0.2930814050090531
Validation error:  0.2571772306777407
[-4.31399250e+04  4.48560306e+04 -4.29180568e+18  1.38059226e+04
 -1.08450215e+04  6.80905312e+04  5.46630741e+04  1.50847864e+04
  9.99250861e+04  3.82376191e+18  2.15825475e+18 -8.26750500e+04
  6.14447839e+02 -2.55271270e+04  8.40162829e+04 -2.35044603e+04
  3.30742683e+04 -1.67546336e+04]
536955.1831843752


Is there a way to make a NN network learn a linear model?

Let's first check what is the loss used by MLPRegressor...

In [10]:
#let's write the code to learn a linear model with NN: how? 


#let's define the model
mlp_lr = MLPRegressor(hidden_layer_sizes=(1, ), solver='lbfgs', random_state=numero_di_matricola, activation='identity')

#let's learn the model on training data
mlp_lr.fit(Xtrain_scaled, Ytrain)

#let's print the error (1 - R^2) on training data
print("Training error: ", 1. - mlp_lr.score(Xtrain_scaled, Ytrain))

#let's print the error (1 - R^2) on validation data
print("Validation error: ", 1. - mlp_lr.score(Xval_scaled, Yval))

#let's print the coefficients of the model for the input nodes (but not the bias)
print(mlp_lr.coefs_)

#let's print the coefficient for the bias (i.e., the bias)
print(mlp_lr.intercepts_)


Training error:  0.29295671109521426
Validation error:  0.25820045516810886
[array([[ -61.726915  ],
       [  77.31319349],
       [ 124.10309685],
       [  23.42628603],
       [ -20.55256929],
       [ 111.60161602],
       [  89.64371674],
       [  24.5959074 ],
       [ 161.09613056],
       [ 126.22872293],
       [  24.41881915],
       [-132.95553575],
       [   1.85202943],
       [ -41.41625462],
       [ 135.83768791],
       [ -39.22387914],
       [  54.79249128],
       [ -27.35141299]]), array([[611.56940786]])]
[array([876.2139533]), array([580.86354434])]


Note that there is an $\ell_2$ regularization term in MLPRegressor. What about making it smaller?

In [11]:
#let's write the code to learn a linear model with NN: how? 


#let's define the model
mlp_lr_noreg = MLPRegressor(hidden_layer_sizes=(1, ), solver='lbfgs', random_state=numero_di_matricola, activation='identity', alpha = 1e-20)

#let's learn the model on training data
mlp_lr_noreg.fit(Xtrain_scaled, Ytrain)

#let's print the error (1 - R^2) on training data
print("Training error: ", 1. - mlp_lr_noreg.score(Xtrain_scaled, Ytrain))

#let's print the error (1 - R^2) on validation data
print("Validation error: ", 1. - mlp_lr_noreg.score(Xval_scaled, Yval))

#let's print the coefficients of the model for the input nodes (but not the bias)
print(mlp_lr_noreg.coefs_)

#let's print the coefficient for the bias (i.e., the bias)
print(mlp_lr_noreg.intercepts_)


Training error:  0.29295671109521415
Validation error:  0.2582004551681061
[array([[ -61.726915  ],
       [  77.31319349],
       [ 124.10309685],
       [  23.42628603],
       [ -20.55256929],
       [ 111.60161602],
       [  89.64371674],
       [  24.5959074 ],
       [ 161.09613056],
       [ 126.22872293],
       [  24.41881915],
       [-132.95553575],
       [   1.85202943],
       [ -41.41625462],
       [ 135.83768791],
       [ -39.22387914],
       [  54.79249128],
       [ -27.35141299]]), array([[611.56940787]])]
[array([876.2139533]), array([580.86354434])]


## More Complex NNs

Let's try more complex NN, for example increasing the number of nodes in the only hidden layer, or increasing the number of hidden layers.

Let's build a NN with 2 nodes in the only hidden layer

In [12]:
#let's build a NN with 2 nodes in the only hidden layer

#let's define the model
mlp_1h2n = MLPRegressor(hidden_layer_sizes=(2, ), solver='lbfgs', random_state=numero_di_matricola )

#let's learn the model on training data
mlp_1h2n.fit(Xtrain_scaled, Ytrain)

#let's print the error (1 - R^2) on training data
print("Training error: ", 1. - mlp_1h2n.score(Xtrain_scaled, Ytrain))

#let's print the error (1 - R^2) on validation data
print("Validation error: ", 1. - mlp_1h2n.score(Xval_scaled, Yval))

#let's print the coefficients of the model for the input nodes (but not the bias)
print(mlp_1h2n.coefs_)

#let's print the coefficient for the bias (i.e., the bias)
print(mlp_1h2n.intercepts_)

Training error:  0.23352689481997224
Validation error:  0.21808319059638492
[array([[ 200.34711416, -200.38783731],
       [-167.95442673,  246.14511148],
       [-272.95572267,  330.62924862],
       [  -7.69669459,  -32.15644844],
       [ 223.2425765 , -136.57641543],
       [-268.51475304,  185.82344487],
       [-199.42402425,  216.92819448],
       [   0.90950512,   56.00110532],
       [ -88.15966565,  394.01863446],
       [-298.65711894,  347.42791614],
       [ -13.82126302,   42.39730829],
       [ 227.11573974, -400.95713087],
       [  27.89296697,  -22.24773761],
       [ 121.43713008, -178.61671533],
       [ -53.52209401,  349.66069475],
       [  90.33391074, -156.62347072],
       [ -24.22328213,  146.23802516],
       [ 219.77683904, -185.34001779]]), array([[211.52307056],
       [350.38337891]])]
[array([759.25601625, 910.08558581]), array([383.26349046])]


Let's build a NN with 5 nodes in the only hidden layer

In [13]:
#let's build a NN with 5 nodes in the only hidden layer

#let's define the model
mlp_1h5n = MLPRegressor(hidden_layer_sizes=(5, ), solver='lbfgs', random_state=numero_di_matricola )
#mlp_1h5n = MLPRegressor(hidden_layer_sizes=(5, ), solver='lbfgs', random_state=numero_di_matricola,  max_iter=2000 )

#let's learn the model on training data
mlp_1h5n.fit(Xtrain_scaled, Ytrain)

#let's print the error (1 - R^2) on training data
print("Training error: ", 1. - mlp_1h5n.score(Xtrain_scaled, Ytrain))

#let's print the error (1 - R^2) on validation data
print("Validation error: ", 1. - mlp_1h5n.score(Xval_scaled, Yval))

#let's print the coefficients of the model for the input nodes (but not the bias)
print(mlp_1h5n.coefs_)

#let's print the coefficient for the bias (i.e., the bias)
print(mlp_1h5n.intercepts_)

Training error:  0.148497765728973
Validation error:  0.21174835603449405
[array([[ 1.01192552e+02, -9.16651795e+00, -5.52338752e+01,
         3.45134231e+01, -4.12171233e+01],
       [-6.75877763e+01,  5.72898439e+01,  1.60935657e+02,
        -4.66447372e-01,  2.06912366e+01],
       [-1.61507916e+01, -5.86525144e+00,  8.52125393e+01,
        -1.08196854e+02,  1.02465553e+02],
       [-5.04516904e+01, -6.72918535e+01,  5.21079890e+01,
         1.78589304e+01,  2.22944378e+01],
       [-1.64202149e+02,  2.45900412e+00,  1.58618880e+01,
         2.60337998e+02, -8.32565638e+01],
       [-1.34013215e+02,  1.20330471e+02,  1.00403750e+02,
        -8.08746591e+01, -3.69143298e+01],
       [-1.33111075e+02,  3.16534489e+01,  4.61073062e+01,
        -1.08772485e+02,  7.14656672e+01],
       [ 9.14474603e+00,  1.33055102e+02,  9.28196914e+01,
         6.69838364e+01, -1.94296836e+01],
       [-1.42407764e+02,  1.25930726e+02,  6.24086343e+01,
         3.58246782e+01,  1.19580819e+02],
       

c:\Users\feder\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:549: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


Note that with a smaller number of iterations we had a larger error on training set but a smaller error on validation data -> "early stopping is a form of regularization"

Let's build a NN with 10 nodes in the only hidden layer

In [14]:
#let's build a NN with 10 nodes in the only hidden layer

#let's define the model
mlp_1h10n = MLPRegressor(hidden_layer_sizes=(10, ), solver='lbfgs', random_state=numero_di_matricola )
#mlp_1h5n = MLPRegressor(hidden_layer_sizes=(5, ), solver='lbfgs', random_state=numero_di_matricola,  max_iter=2000 )

#let's learn the model on training data
mlp_1h10n.fit(Xtrain_scaled, Ytrain)

#let's print the error (1 - R^2) on training data
print("Training error: ", 1. - mlp_1h10n.score(Xtrain_scaled, Ytrain))

#let's print the error (1 - R^2) on validation data
print("Validation error: ", 1. - mlp_1h10n.score(Xval_scaled, Yval))

#let's print the coefficients of the model for the input nodes (but not the bias)
print(mlp_1h10n.coefs_)

#let's print the coefficient for the bias (i.e., the bias)
print(mlp_1h10n.intercepts_)

Training error:  0.09648940765735636
Validation error:  0.16413837672914344
[array([[-5.71756512e+01, -3.46380819e+01, -1.17905569e+02,
        -5.28626008e+01, -4.60580291e+01, -2.04724471e+01,
         3.14560505e+01,  8.69824156e+01,  7.47755869e+00,
         1.71918891e+02],
       [ 7.90932422e+01, -3.55424669e+00,  1.97761395e+02,
         4.47316498e+01, -3.37271337e+01,  1.10904175e+02,
        -9.55093228e+01,  1.29956746e+02, -6.12690022e+01,
        -1.34131421e+02],
       [ 2.97526070e+01,  8.86306472e+01,  7.85065778e+01,
         5.39202538e+01,  8.92660494e+01,  2.76843858e+01,
         7.02735949e+01, -5.64849761e+01,  3.87296761e+01,
        -3.06748913e+01],
       [-3.02804160e+01,  4.80719694e+01, -2.91318698e+01,
         3.81362552e+01,  1.62207422e+00, -2.43037503e+00,
        -7.26291829e+01,  1.04895354e+02, -7.51765265e+01,
        -8.41831877e+00],
       [ 1.09941595e+02, -1.32752322e+02, -5.85084034e+01,
        -1.22292599e+02, -1.02660860e+02,  1.0203150

c:\Users\feder\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:549: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


Let's build a NN with 100 nodes in the only hidden layer. Note that this is the default!

In [15]:
#let's build a NN with 100 nodes in the only hidden layer


#let's define the model
mlp_1h100n = MLPRegressor(hidden_layer_sizes=(100, ), solver='lbfgs', random_state=numero_di_matricola )

#let's learn the model on training data
mlp_1h100n.fit(Xtrain_scaled, Ytrain)

#let's print the error (1 - R^2) on training data
print("Training error: ", 1. - mlp_1h100n.score(Xtrain_scaled, Ytrain))

#let's print the error (1 - R^2) on validation data
print("Validation error: ", 1. - mlp_1h100n.score(Xval_scaled, Yval))

#let's print the coefficients of the model for the input nodes (but not the bias)
print(mlp_1h100n.coefs_)

#let's print the coefficient for the bias (i.e., the bias)
print(mlp_1h100n.intercepts_)

Training error:  0.0319051406662455
Validation error:  0.25036082662669157
[array([[ -18.94304082,   -6.70923412,    3.45897283, ...,   35.98744405,
          11.81401137,    4.57930877],
       [ -20.86346717,  -61.59832987,  -71.27498445, ..., -105.31637912,
         -78.21085399,   -6.62602938],
       [  37.59540596,   52.95612071,  -73.50032062, ...,   -1.24071395,
          75.68539773,   41.68042155],
       ...,
       [-307.73700692,  -53.87204983,  -90.87783897, ..., -141.24503853,
          37.43076098, -334.35566932],
       [ 178.03212677,  -22.81824994,  182.60311888, ...,  176.97786211,
         -37.39394643,    2.65481682],
       [-386.01097596, -202.70111933,  -21.74051091, ...,    6.51243215,
         -21.07150601, -106.45083058]]), array([[ -34.45577825],
       [-212.66761878],
       [ 244.21660387],
       [ -80.01612047],
       [ 136.55414191],
       [ 210.66091622],
       [ -85.82651357],
       [ 566.4872013 ],
       [  37.07656643],
       [ 245.02052931]

c:\Users\feder\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:549: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


Let's try 2 layers, 1 node each

In [16]:
#let's build a NN with 2 hidden layers, 1 node each

#let's define the model
mlp_2h1n1n = MLPRegressor(hidden_layer_sizes=(1,1 ), solver='lbfgs', random_state=numero_di_matricola)
#after warning shows up
#mlp_2h2n2n = MLPRegressor(hidden_layer_sizes=(1,1 ), solver='lbfgs', random_state=numero_di_matricola, max_iter=2000)

#let's learn the model on training data
mlp_2h1n1n.fit(Xtrain_scaled, Ytrain)

#let's print the error (1 - R^2) on training data
print("1 - coefficient of determination on training data:"+str(1 - mlp_2h1n1n.score(Xtrain_scaled,Ytrain)))

#let's print the error (1 - R^2) on validation data
print("1 - coefficient of determination on validation data:"+str(1 - mlp_2h1n1n.score(Xval_scaled,Yval)))

#let's print the coefficients of the model for the input nodes (but not the bias)
print(mlp_2h1n1n.coefs_)

#let's print the coefficient for the bias (i.e., the bias)
print(mlp_2h1n1n.intercepts_)

1 - coefficient of determination on training data:0.2590073361942665
1 - coefficient of determination on validation data:0.2434130279211909
[array([[-11.48912015],
       [ 21.17055323],
       [ 22.76411195],
       [ 10.73246099],
       [ -4.30244111],
       [ 18.6473214 ],
       [ 15.53921545],
       [  6.34288444],
       [ 39.91123524],
       [ 24.84341392],
       [  2.52430101],
       [-32.08441276],
       [ -1.2018783 ],
       [-18.70686048],
       [ 40.52380366],
       [-25.55717232],
       [ 15.12041708],
       [-14.34407581]]), array([[8.13890817]]), array([[443.40469451]])]
[array([30.62642389]), array([718.27969296]), array([101.13257123])]


Let's try 2 layers, 2 nodes each

In [17]:
#let's build a NN with 2 layers, 2 nodes each

#let's define the model
mlp_2h2n = MLPRegressor(hidden_layer_sizes=(2,2, ), solver='lbfgs', random_state=numero_di_matricola )

#let's learn the model on training data
mlp_2h2n.fit(Xtrain_scaled, Ytrain)

#let's print the error (1 - R^2) on training data
print("Training error: ", 1. - mlp_2h2n.score(Xtrain_scaled, Ytrain))

#let's print the error (1 - R^2) on validation data
print("Validation error: ", 1. - mlp_2h2n.score(Xval_scaled, Yval))

#let's print the coefficients of the model for the input nodes (but not the bias)
print(mlp_2h2n.coefs_)

#let's print the coefficient for the bias (i.e., the bias)
print(mlp_2h2n.intercepts_)

Training error:  1.0
Validation error:  1.0006494985858037
[array([[  1298.42505828,   3630.4757554 ],
       [ -4779.07967288,   7310.11425883],
       [-14755.45357693,  12836.32577226],
       [   935.03737434,   -135.91753605],
       [ -4362.82837819,   5265.09644754],
       [ -6654.24405888,   2147.01332316],
       [-17160.93827266,  10694.15073316],
       [-24768.62733118,   4844.78981221],
       [ -6093.73131655,   9603.53352438],
       [ -9312.25569691,   9750.72297238],
       [-12843.68696299,   8250.78675234],
       [ 15930.88180455,  -6389.5302134 ],
       [ -5666.94949579,   5139.85135624],
       [  1250.98287833,    717.31978561],
       [-17205.6946984 ,   6482.16177771],
       [  2096.64527185,  -2704.89081656],
       [ -2295.24244786,   6253.74455925],
       [  2804.35678291,    245.96426135]]), array([[-1.04678561e+03, -2.33866124e+04],
       [-4.29110819e+00, -1.75198459e+04]]), array([[ 1631.45930753],
       [93629.141364  ]])]
[array([-39783.01081832,

Let's try 2 layers, 10 nodes each

In [18]:
#let's build a NN with 2 layers, 10 nodes each

#let's define the model
mlp_2h10n10n = MLPRegressor(hidden_layer_sizes=(10,10 ), solver='lbfgs', random_state=numero_di_matricola)

#let's learn the model on training data
mlp_2h10n10n.fit(Xtrain_scaled, Ytrain)

#let's print the error (1 - R^2) on training data
print("1 - coefficient of determination on training data:"+str(1 - mlp_2h10n10n.score(Xtrain_scaled,Ytrain)))

#let's print the error (1 - R^2) on validation data
print("1 - coefficient of determination on validation data:"+str(1 - mlp_2h10n10n.score(Xval_scaled,Yval)))

#let's print the coefficients of the model for the input nodes (but not the bias)
print(mlp_2h10n10n.coefs_)

#let's print the coefficient for the bias (i.e., the bias)
print(mlp_2h10n10n.intercepts_)

1 - coefficient of determination on training data:0.10017767273094247
1 - coefficient of determination on validation data:0.29595524264949646
[array([[-3.98695547e+01, -2.84178524e+00, -3.23388775e+00,
         1.36498567e+01,  1.21498009e+00,  3.87107437e+01,
        -9.54085467e-01,  1.28715827e+01,  1.08436058e-01,
         2.33689301e+01],
       [-1.74201092e+00,  1.53730253e+01,  2.03580414e+01,
         4.33558269e+00, -1.54386203e+01,  1.13533480e+01,
        -2.06867256e+01,  9.73568792e-01, -1.97535783e+01,
         2.42705136e+01],
       [ 1.18889512e+01,  9.13365828e+00, -2.78769732e+00,
         1.17232188e+00,  2.11838854e+00, -9.36287951e+00,
         9.53588256e+00,  1.51262884e+01, -1.26960466e+01,
        -8.82653204e+00],
       [ 4.22380190e+00,  1.22915629e+01, -2.25838424e+01,
         7.01780647e+00, -1.76391748e+01,  2.47787382e+01,
         5.61919889e+00, -5.49962104e+00, -9.29574123e+00,
         3.26609814e+01],
       [-2.84172441e+01, -2.11017067e+01,  3.

c:\Users\feder\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:549: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


Let's try 2 layers, 100 nodes each

In [19]:
#let's build a NN with 2 layers, 100 nodes each

#let's define the model
mlp_2h100n100n = MLPRegressor(hidden_layer_sizes=(100,100 ), solver='lbfgs', random_state=numero_di_matricola)

#let's learn the model on training data
mlp_2h100n100n.fit(Xtrain_scaled, Ytrain)

#let's print the error (1 - R^2) on training data
print("1 - coefficient of determination on training data:"+str(1 - mlp_2h100n100n.score(Xtrain_scaled,Ytrain)))

#let's print the error (1 - R^2) on validation data
print("1 - coefficient of determination on validation data:"+str(1 - mlp_2h100n100n.score(Xval_scaled,Yval)))

#let's print the coefficients of the model for the input nodes (but not the bias)
print(mlp_2h100n100n.coefs_)

#let's print the coefficient for the bias (i.e., the bias)
print(mlp_2h100n100n.intercepts_)

1 - coefficient of determination on training data:0.021926618551395105
1 - coefficient of determination on validation data:0.3027891456738775
[array([[ -8.07373452,   2.48112682,   3.72629452, ...,  -2.35572895,
          0.11664826,  -4.4739852 ],
       [ 14.69799642, -13.68833811,   1.77313337, ...,   8.10668659,
          1.58587943,  -0.83111523],
       [ -2.92738924, -11.0651268 ,   6.6031689 , ...,   2.03370413,
          8.13280235,   4.59196146],
       ...,
       [-20.26960083,  -8.56903481,   7.83060203, ...,  -1.58964287,
        -20.59473701,  -9.42896324],
       [-20.44810458,  23.74577985,  -4.29557866, ...,   6.84870183,
         14.93858852,  -7.6062445 ],
       [ 12.42773877,   3.06532615,  -4.84591698, ..., -20.24550646,
        -28.54966097,  -8.64585581]]), array([[-1.79729992e+01, -6.41416557e-01, -5.80587110e-01, ...,
        -3.05353767e+00, -3.49326684e+00, -2.94807874e-01],
       [ 7.55363938e+00, -3.73879527e-02, -2.93225024e-01, ...,
         2.67287195

c:\Users\feder\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:549: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


So it seems that 1 layer (and default number of iterations) works best for this dataset. Let's try 5-fold cross-validation with number of nodes in the hidden layer between 1 and 20.
Note that we use train and validation data together, since we are doing cross-validation.

In [20]:
from sklearn.model_selection import GridSearchCV

mlp_cv = MLPRegressor()
param_grid = {'hidden_layer_sizes': [i for i in range(1,21)],
              'activation': ['relu'],
              'solver': ['lbfgs'], 
              'random_state': [numero_di_matricola]
             }
mlp_GS = GridSearchCV(mlp_cv, param_grid=param_grid, 
                   cv=5, verbose=True)
mlp_GS.fit(Xtrain_and_val_scaled, Ytrain_and_val)

Fitting 5 folds for each of 20 candidates, totalling 100 fits


c:\Users\feder\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:549: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
c:\Users\feder\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:549: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
c:\Users\feder\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:549: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITE

GridSearchCV(cv=5, estimator=MLPRegressor(),
             param_grid={'activation': ['relu'],
                         'hidden_layer_sizes': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10,
                                                11, 12, 13, 14, 15, 16, 17, 18,
                                                19, 20],
                         'random_state': [2074282], 'solver': ['lbfgs']},
             verbose=True)

Now let's check what is the best parameter, and compare the best NNs with the linear model (learned on train and validation) on test data.

In [21]:
#let's print the best model according to grid search
print("Best model: ",mlp_GS.best_estimator_)
#let's print the error 1-R^2 for the best model
print("Error (1-R^2) of best model: ",1. - mlp_GS.best_score_)

Best model:  MLPRegressor(hidden_layer_sizes=18, random_state=2074282, solver='lbfgs')
Error (1-R^2) of best model:  0.21031466908259344


Let compare the error of the best NN on train and validation and on test data.

In [22]:
print("Error best model on train and validation: ",1. - mlp_GS.best_estimator_.score(Xtrain_and_val_scaled,Ytrain_and_val))
print("Error best model on test data: ",1. - mlp_GS.best_estimator_.score(Xtest_scaled,Ytest))

Error best model on train and validation:  0.07619186240044695
Error best model on test data:  0.2376929248934987


Now let's learn the linear model on train and validation, and get error (1-R^2) on train and validation and on test data.

In [23]:
#LR the linear regression model
LR = linear_model.LinearRegression()

#fit the model on training data
LR.fit(Xtrain_and_val_scaled, Ytrain_and_val)

print("1 - coefficient of determination on training data:"+str(1 - LR.score(Xtrain_and_val_scaled,Ytrain_and_val)))
print("1 - coefficient of determination on test data:"+str(1 - LR.score(Xtest_scaled,Ytest)))

1 - coefficient of determination on training data:0.28061435677273594
1 - coefficient of determination on test data:0.3018066935576518


Note: MLPRegressor has several other parameters!